# Gym Startup
### Trevor Fernstrom

### 1. Introduction

#### 1.1 Background

Fitness is increasingly becoming apart of peoples lives as science continues to emphasis the importance of exercise for everyone. According to International Health, Racquet & Sportsclub Association (IHRSA), the gym industry in the U.S. made [32.3 billion](https://www.wellnesscreatives.com/gym-market-statistics/#market) in revenue in 2018. There are over [105,000 gym/fitness clubs in the U.S.](https://www.ibisworld.com/industry-statistics/number-of-businesses/gym-health-fitness-clubs-united-states/) and this number has been slowly increasing over the past several years. 


#### 1.2 Problem

Every entrepreneur is faced with the problem of how they are going to sell their product/service to their desired customers. Goods and services can be purchased either online or in-person. For someone wanting to open a retail business, choosing store location is critical in getting good business and sustaining operations. Several factors must be considered when opening a gym. The variables included in this analysis are state rent, state utilities, number of gyms, population, and the relative easiness to open a business. Using some of the variables mentioned, choosing the state and city to operate is the first decision. Then the problem becomes, given the city of interest, where would be the best location to open a gym?

### 2. Data

#### 2.1 Data Description

To select the best location, we need to narrow our search down to a specific state. To do this, data was collected on state utilities, building cost, level of physical activity, and the overall easiness to start a business. The utilities data can be found [here](https://www.move.org/utility-bills-101/), which contains the average cost of electricity, natural gas, and water by state in the United States. To get a sense of prices of building in the United States, average mortgage prices for homes was used and can be found [here](https://www.experian.com/blogs/ask-experian/research/median-home-values-by-state/). The data for the level of physical activity in each state can be found [here](https://www.cdc.gov/physicalactivity/data/inactivity-prevalence-maps/index.html). It is important to mention that this data was self-reported and measures physical inactivity. The last variable, overall easiness to open a business, can be found [here](https://www.seekcapital.com/blog/best-states-to-start-business/). The ranks determined in this data take into account 21 different indicators that influence the starting of a business such as availability of labor, tax climate, business survival rate, and etc.

#### 2.2 Data Cleaning

Data was scraped from these websites, cleaned, and then combined into one table. Missing values were not an issue as there was data for all 50 states in each table. However, U.S. territories were removed from tables that contained them. This is because not all the tables contained these territories and the fact that the states are what is of interest. Some of the data that was retrieved was not important for the analysis and was removed. The data was then ranked based on their values. Utilities and mortgage were ranked based on their monetary values from least to greatest, having the smaller value being the higher rank. Ranking was already provided for the business startup data. The physical activity data was ranked based on inactivity prevalence. States with smaller physical inactivity prevalence were ranked higher. Lastly, data from Foursquare was used to determine the specific neighborhood that would be best to open a gym based on gym prevalence in that area. 

#### 2.3 Data Preparation

In [1]:
!pip install beautifulsoup4
!pip install lxml
!conda install -c conda-forge folium=0.5.0 --yes
print("Libraries Installed!")

Solving environment: done

# All requested packages already installed.

Libraries Installed!


In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import json
from pandas.io.json import json_normalize

In [3]:
Utilities_html = 'https://www.move.org/utility-bills-101/'
Rent_html = 'https://www.experian.com/blogs/ask-experian/research/median-home-values-by-state/'
BusinessOpen = 'https://www.seekcapital.com/blog/best-states-to-start-business/'
Exercise_html = 'https://www.cdc.gov/physicalactivity/data/inactivity-prevalence-maps/index.html'
utilities_html_content = requests.get(Utilities_html).text
rent_html_content = requests.get(Rent_html).text
open_html_content = requests.get(BusinessOpen).text
exercise_html_content = requests.get(Exercise_html).text

In [4]:
utilities_ParseData = BeautifulSoup(utilities_html_content, "lxml")
rent_ParseData = BeautifulSoup(rent_html_content, "lxml")
open_ParseData = BeautifulSoup(open_html_content, "lxml")
exercise_ParseData = BeautifulSoup(exercise_html_content, "lxml")

In [5]:
utilities_htmlTable = utilities_ParseData.find_all('table', attrs={'class': 'scrolling-table__values__table', 'style': 'width:630px; transform:translate3d(-630px,0,0);'}) # Find the Table
rent_htmlTable = rent_ParseData.find_all('table', attrs={'class': 'two-headers-tbl scroll'})
exercise_htmlTable = exercise_ParseData.find_all('table', attrs={'class': 'table-bordered table table-striped table-striped-light opt-in show-more-div-138 table-sm fs08'})

In [6]:
Overall_Rank = []
Title_Rank = []
Open_State = []

for row in open_ParseData.find_all('li'):
    #if row.find_all('b')== 'Overall Rank: '
    titlecells = row.find_all('b')
    cells = row.find_all('span')
    if len(titlecells)==1:
        Title_Rank.append(titlecells[0].find(text=True).replace('\n', ' ').strip())
        
    if len(cells)==1:
        Overall_Rank.append(cells[0].find(text=True).replace('\n', ' ').strip())

for row in open_ParseData.find_all('h2'):
    Open_State.append(row.find(text=True).replace('\n', ' ').strip())

In [7]:
State_data = pd.DataFrame(Open_State, columns=['State'])
Rank_data = pd.DataFrame(Overall_Rank, columns=['Startup Rank'])
Title_data = pd.DataFrame(Title_Rank, columns=['Title Rank'])
State_data = State_data[4:54]
State_data = State_data.reset_index(drop=True)
Rank_data = Rank_data[4:160]
Rank_data = Rank_data.reset_index(drop=True)
Title_data['Startup Rank'] = Rank_data
Title_data = Title_data[Title_data['Title Rank'] == "Overall Rank:"]
Title_data = Title_data.reset_index(drop=True)
Title_data['State'] = State_data
Title_data = Title_data.replace(':', 46)
Startup_data = Title_data[['State']]
Title_data = Title_data[['Startup Rank']].astype(float)
Startup_data['Startup Rank'] = Title_data[['Startup Rank']]
Startup_data.head()

,State,Startup Rank
0,Alabama,46.0
1,Alaska,22.0
2,Arizona,15.0
3,Arkansas,43.0
4,California,5.0


In [8]:
Exercise_State = []
Prevalence = []
ConfidenceInterval = []

for rows in exercise_htmlTable[0].find_all('tr'):
    col1 = rows.find_all('th')
    Exercise_State.append(col1[0].find(text=True))
for rows2 in exercise_htmlTable[0].find_all('tr'):
    col2 = rows2.find_all('td')
    if len(col2)==2:
        Prevalence.append(col2[0].find(text=True).replace('\n', ' ').strip())
        ConfidenceInterval.append(col2[1].find(text=True).replace('\n', ' ').strip())

In [9]:
Exercise_data = pd.DataFrame(Exercise_State, columns=['State'])
Exercise_data = Exercise_data[2:55]
Exercise_data = Exercise_data.reset_index(drop=True)
Exercise_data['Prevalence'] = Prevalence
Exercise_data['ConfidenceInterval'] = ConfidenceInterval
Exercise_data = Exercise_data.sort_values(by=['Prevalence'])
Exercise_data['Exercise Rank'] = Exercise_data['Prevalence'].rank()
Exercise_data.head()

,State,Prevalence,ConfidenceInterval,Exercise Rank
5,Colorado,17.3,"(16.8, 17.8)",1.0
47,Washington,18.3,"(17.9, 18.8)",2.0
44,Utah,18.6,"(18.1, 19.0)",3.0
37,Oregon,19.2,"(18.5, 19.8)",4.0
8,District of Columbia,19.8,"(18.9, 20.8)",5.0


In [10]:
Rent_State = []
FICO = []
HomeValue = []
Mortgage = []
Difference = []

for row in rent_htmlTable[0].find_all('tr'):
    cells = row.find_all('td')
    if len(cells)==5:
        Rent_State.append(cells[0].find(text=True).replace('\n', ' ').strip())
        FICO.append(cells[1].find(text=True).replace('\n', ' ').strip())
        HomeValue.append(cells[2].find(text=True).replace('\n', ' ').strip())
        Mortgage.append(cells[3].find(text=True).replace('\n', ' ').strip())
        Difference.append(cells[4].find(text=True).replace('\n', ' ').strip())

In [11]:
Rent_data = pd.DataFrame(Rent_State, columns=['State'])
Rent_data['FICO'] = FICO
Rent_data['HomeValue'] = HomeValue
Rent_data['Mortgage'] = Mortgage
Rent_data = Rent_data[['State', 'FICO','HomeValue','Mortgage']]
Rent_data['FICO'] = Rent_data['FICO'].str.replace('$', '')
Rent_data['HomeValue'] = Rent_data['HomeValue'].str.replace('$', '')
Rent_data['Mortgage'] = Rent_data['Mortgage'].str.replace('$', '')
Rent_data['FICO'] = Rent_data['FICO'].str.replace(',', '')
Rent_data['HomeValue'] = Rent_data['HomeValue'].str.replace(',', '')
Rent_data['Mortgage'] = Rent_data['Mortgage'].str.replace(',', '')
Rent_data[['FICO', 'HomeValue','Mortgage']] = Rent_data[['FICO', 'HomeValue','Mortgage']].astype(int)
Rent_data = Rent_data[['State', 'Mortgage']]
Rent_data = Rent_data.sort_values(by=['Mortgage'])
Rent_data['Rent Rank'] = Rent_data['Mortgage'].rank()
Rent_data.head()

,State,Mortgage,Rent Rank
50,West Virginia,110464,1.0
40,Indiana,120567,2.0
47,Mississippi,122107,3.0
44,Ohio,122939,4.0
41,Kentucky,126485,5.0


In [12]:
Utilities_State = []
Electricity = []
NaturalGas = []
Water = []
Internet = []
Cable = []
Total = []

for row in utilities_htmlTable[0].find_all('tr'):
    cells = row.find_all('td')
    if len(cells)==7:
        Utilities_State.append(cells[0].find(text=True).replace('\n', ' ').strip())
        Electricity.append(cells[1].find(text=True).replace('\n', ' ').strip())
        NaturalGas.append(cells[2].find(text=True).replace('\n', ' ').strip())
        Water.append(cells[3].find(text=True).replace('\n', ' ').strip())
        Internet.append(cells[4].find(text=True).replace('\n', ' ').strip())
        Cable.append(cells[5].find(text=True).replace('\n', ' ').strip())
        Total.append(cells[6].find(text=True).replace('\n', ' ').strip())

In [13]:
Utilities_data = pd.DataFrame(Utilities_State, columns=['State'])
Utilities_data['Electricity'] = Electricity
Utilities_data['NaturalGas'] = NaturalGas
Utilities_data['Water'] = Water
Utilities_data['Internet'] = Internet
Utilities_data['Cable'] = Cable
Utilities_data['Total'] = Total
Utilities_data = Utilities_data[Utilities_data['State']!= "State"] 
Utilities_data = Utilities_data[['State', 'Electricity','NaturalGas','Water']]
Utilities_data['Electricity'] = Utilities_data['Electricity'].str.replace('$', '')
Utilities_data['NaturalGas'] = Utilities_data['NaturalGas'].str.replace('$', '')
Utilities_data['Water'] = Utilities_data['Water'].str.replace('$', '')
Utilities_data[['Electricity', 'NaturalGas','Water']] = Utilities_data[['Electricity', 'NaturalGas','Water']].astype(float)
Utilities_data['Total'] = Utilities_data['Electricity'] + Utilities_data['NaturalGas'] + Utilities_data['Water']
Utilities_data = Utilities_data.reset_index(drop=True)
Utilities_data = Utilities_data.sort_values(by=['Total'])
Utilities_data['Utilities Rank'] = Utilities_data['Total'].rank()
Utilities_data.head()

,State,Electricity,NaturalGas,Water,Total,Utilities Rank
50,New Mexico,79.16,50.00,70.39,199.55,1.0
49,Utah,81.65,47.99,70.39,200.03,2.0
48,Colorado,82.47,50.57,70.39,203.43,3.0
47,Montana,94.75,42.60,70.39,207.74,4.0
46,Idaho,100.38,41.76,70.39,212.53,5.0


In [14]:
State_Ranks =  pd.DataFrame(Utilities_data['State'])
State_Ranks['Utilities Rank'] = Utilities_data['Utilities Rank']
State_Ranks = State_Ranks.reset_index(drop=True)
State_Ranks = State_Ranks[0:51]
State_Ranks = State_Ranks.join(Rent_data.set_index('State'),on = 'State')
State_Ranks = State_Ranks[State_Ranks['State'] != "District of Columbia"]
State_Ranks = pd.merge(State_Ranks, Startup_data, left_on = "State", right_on = "State")
State_Ranks = pd.merge(State_Ranks, Exercise_data, left_on = "State", right_on = "State")
State_Ranks = State_Ranks[['State', 'Utilities Rank', 'Rent Rank', 'Startup Rank', 'Exercise Rank']]

In [15]:
State_Ranks['Average'] = State_Ranks.mean(axis=1)

In [16]:
State_Ranks.sort_values(by=['Average']).head()

,State,Utilities Rank,Rent Rank,Startup Rank,Exercise Rank,Average
1,Utah,2.0,37.0,1.0,3.0,10.750
4,Idaho,5.0,25.0,7.0,11.0,12.000
2,Colorado,3.0,47.0,4.0,1.0,13.750
18,South Dakota,19.0,18.0,11.0,14.5,15.625
3,Montana,4.0,31.0,12.0,16.0,15.750
